In [1]:
import wandb

api = wandb.Api(overrides={"base_url": "https://microsoft-research.wandb.io/"})
#run = api.run("/msr-mtl/text-games-benchmark/runs/7e1kyl3h")
runs = api.runs(
    "msr-mtl/text-games-benchmark",
    filters={
        "state": "finished",
        "config.model": {"$in": [
            "gpt-4o",
            "phi-3-medium-128k-instruct-1",
            "microsoft/Phi-3.5-mini-instruct",
            "meta-llama-3-70b-instruct-4",
            "microsoft/Phi-3-mini-128k-instruct",
            "random-agent",
        ]}
    }
)

#print(run.history())
print(len(runs))


321


In [39]:
print(runs[0].config)
print(runs[0].summary)

{'llm': 'gcr_phi', 'game': '905.z5', 'seed': 1234, 'model': 'phi-3-medium-128k-instruct-1', 'context': 100, 'temperature': 0, 'conversation': False, 'admissible_commands': True}
{'Context': 100, 'Final score': 0, 'Max Normalized Score': 0, 'Max Score': 1, 'Moves': 120, 'Normalized Score': 0, 'Score': 0, 'Step': 1000, 'Total steps': 1000, '_runtime': 881.5551450252533, '_step': 1001, '_timestamp': 1724991275.505299, '_wandb': {'runtime': 881}, 'log_table': {'_latest_artifact_path': 'wandb-client-artifact://8h36l8rfej07i86jnws36mrdj3a5dg8d0r3xab3tupkcrd9z94hcxef48gfngfshjhyk5m7fol06iu0h07qtmn6c8vob7c1hp5y8n11915xpzlvaya5g3ax5po8zl9u3:latest/log_table.table.json', '_type': 'table-file', 'artifact_path': 'wandb-client-artifact://udpuoe9wyevsogq5fser0vusqlprgzm7su8r4a31qwruei4h4i1ag56r03dh3vvvu5fitnd4mp61pwzff2jek9ojskca39mpm48rbg1f7shtv194qr86ppxfnkqcyb7u/log_table.table.json', 'ncols': 12, 'nrows': 1000, 'path': 'media/table/log_table_1001_570422b1a24bd26301a7.table.json', 'sha256': '5704

In [54]:
# Load game_infos.tsv
import pandas as pd
game_infos = pd.read_csv("game_infos.tsv", sep="\t")
game_infos

,Filename,Template Action,Solution,Avg. Steps,Stochastic,Dialog,Darkness,Nonstandard Action,Inventory,Difficulty,...,Max Score,Maze,Trivia,Self-Maintenance,Combat,Time,Mechanical Puzzle,Persistence,Stochasticity,Forgiveness
0,905,7,22,22,False,False,False,False,False,Possible,...,NaN,False,False,False,False,False,False,False,False,Polite
1,acorncourt,17,17,8,False,False,False,False,False,Possible,...,30.0,False,False,False,False,False,True,False,False,Merciful
2,advent,107,277,7,True,False,True,True,True,Difficult,...,350.0,True,True,False,True,True,True,True,False,Tough
3,adventureland,23,170,13,True,False,True,False,True,Possible,...,100.0,True,True,False,True,True,True,True,True,Tough
4,afflicted,79,98,7,False,False,True,False,False,Possible,...,75.0,False,False,True,True,True,True,False,False,Polite
5,anchor,1268,531,15,True,True,True,False,False,Extreme,...,100.0,False,True,False,True,True,False,True,False,Cruel
6,awaken,37,57,8,False,False,False,False,False,Possible,...,50.0,False,True,False,True,True,True,True,False,Polite
7,balances,30,122,12,False,False,True,True,False,Difficult,...,51.0,False,True,False,False,False,True,False,False,Polite
8,ballyhoo,310,416,21,False,True,False,False,True,Extreme,...,300.0,True,True,True,False,True,True,False,True,Tough
9,curses,398,816,9,True,True,True,False,True,Extreme,...,550.0,False,True,False,False,True,False,True,True,Polite


In [327]:
# Conver Wandb RUNS to a pandas dataframe
import pandas as pd
import numpy as np
runs_data = []
for run in runs:
    run_data = dict(run.summary)
    run_data["model"] = run.config["model"]
    run_data["game"] = run.config["game"]
    #run_data["game_description"] = game_infos[game_infos["game"] == run.config["game"]]["description"].values[0]
    runs_data.append(run_data)

runs_df = pd.DataFrame(runs_data)

categories = ["Stochastic", "Dialog", "Darkness", "Nonstandard Action", "Inventory", "Maze", "Maze", "Trivia", "Self-Maintenance", "Combat", "Time", "Mechanical Puzzle", "Persistence", "Difficulty"]
ignore_categories = ["Difficulty"]

# Keep game, model, Max Normalized Score
runs_df = runs_df[["game", "model", "Max Normalized Score"]]
runs_df = runs_df.sort_values(by="game")

# For each game add columns with the following games infos: Stochastic, Dialog, Darkness, Nonstandard Action, Inventory, Difficulty
for game in runs_df["game"].unique():
    game_info = game_infos[game_infos["Filename"] == game.split(".")[0]]
    for col in categories:
        runs_df.loc[runs_df["game"] == game, col] = game_info[col].values[0]

runs_df


,game,model,Max Normalized Score,Stochastic,Dialog,Darkness,Nonstandard Action,Inventory,Maze,Trivia,Self-Maintenance,Combat,Time,Mechanical Puzzle,Persistence,Difficulty
0,905.z5,phi-3-medium-128k-instruct-1,0.0,False,False,False,False,False,False,False,False,False,False,False,False,Possible
134,905.z5,gpt-4o,0.0,False,False,False,False,False,False,False,False,False,False,False,False,Possible
213,905.z5,microsoft/Phi-3-mini-128k-instruct,0.0,False,False,False,False,False,False,False,False,False,False,False,False,Possible
8,905.z5,meta-llama-3-70b-instruct-4,0.0,False,False,False,False,False,False,False,False,False,False,False,False,Possible
47,905.z5,microsoft/Phi-3.5-mini-instruct,0.0,False,False,False,False,False,False,False,False,False,False,False,False,Possible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,ztuu.z5,gpt-4o,19.0,True,False,True,False,False,True,True,False,False,True,True,True,Possible
178,ztuu.z5,phi-3-medium-128k-instruct-1,5.0,True,False,True,False,False,True,True,False,False,True,True,True,Possible
265,ztuu.z5,microsoft/Phi-3-mini-128k-instruct,0.0,True,False,True,False,False,True,True,False,False,True,True,True,Possible
118,ztuu.z5,microsoft/Phi-3.5-mini-instruct,0.0,True,False,True,False,False,True,True,False,False,True,True,True,Possible


In [328]:
# Compute the average score for binary features: "Stochastic", "Dialog", "Darkness", "Nonstandard Action", "Inventory"
# and augment the dataframe with these values

radar_df = pd.DataFrame()

for col in categories:
    if col in ignore_categories:
        continue
    radar_df[col] = runs_df.groupby(["model", col])["Max Normalized Score"].mean().unstack()[True]

radar_df.reset_index(inplace=True)

# Convert to three columns format: model, feature, score
radar_df = radar_df.melt(id_vars=["model"], var_name="feature", value_name="score")
radar_df

,model,feature,score
0,gpt-4o,Stochastic,6.066962
1,meta-llama-3-70b-instruct-4,Stochastic,5.299504
2,microsoft/Phi-3-mini-128k-instruct,Stochastic,0.823593
3,microsoft/Phi-3.5-mini-instruct,Stochastic,0.885611
4,phi-3-medium-128k-instruct-1,Stochastic,2.917702
...,...,...,...
67,meta-llama-3-70b-instruct-4,Persistence,6.334042
68,microsoft/Phi-3-mini-128k-instruct,Persistence,2.383700
69,microsoft/Phi-3.5-mini-instruct,Persistence,1.532187
70,phi-3-medium-128k-instruct-1,Persistence,4.361264


In [334]:
# Using plotly build radar chart for each binary feature: "Stochastic", "Dialog", "Darkness", "Nonstandard Action", "Inventory"
import plotly.express as px
import plotly.graph_objects as go


fig = px.line_polar(radar_df,
                    r="score",
                    theta="feature",
                    line_group="model",
                    color="model",
                    line_close=True,
                    #template="plotly_dark",
                    title="Max Normalized Score per Criteria",)
fig.update_layout(polar=dict(radialaxis=dict(range=[0, 100])))

# Move legend to the bottom left
fig.update_layout(legend=dict(
    orientation="v",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
# Hide title
fig.update_layout(title="")
# Hide legen title
fig.update_layout(legend_title="")
# Change figure size to be
fig.write_image("radar_chart.png", width=1600, height=1600)
fig.show()

fig.update_layout(polar=dict(radialaxis=dict(range=[0, 14])))
# Save image to disk
fig.write_image("radar_chart_zoom.png", width=800, height=800)


In [167]:
# Using plotly build radar chart for each binary feature: "Stochastic", "Dialog", "Darkness", "Nonstandard Action", "Inventory"
import plotly.express as px
import plotly.graph_objects as go


fig = px.line_polar(radar_df,
                    r="score",
                    theta="feature",
                    line_group="model",
                    color="model",
                    line_close=True, template="plotly_dark", title="Max Normalized Score per game")
fig.update_layout(polar=dict(radialaxis=dict(range=[0, 100])))
fig.show()

In [189]:
# Using plotly build radar chart for each binary feature: "Stochastic", "Dialog", "Darkness", "Nonstandard Action", "Inventory"
import plotly.express as px
import plotly.graph_objects as go

# Sort runs_df theta according to gpt-4o best max normalized score
runs_df = runs_df.sort_values(by=["model", "Max Normalized Score"], ascending=True)

fig = px.line_polar(runs_df,
                    r="Max Normalized Score",
                    theta="game",
                    line_group="model",
                    color="model",
                    line_close=True, template="plotly_dark", title="Max Normalized Score per game")
fig.update_layout(polar=dict(radialaxis=dict(range=[0, 100])))

fig.show()

In [311]:
# Create a group bar chart with the average score per game for each model
import plotly.express as px
import plotly.graph_objects as go

df = runs_df.copy()

# sort by max score for gpt-4o
#df = df.sort_values(by=["model", "Max Normalized Score"], ascending=True)
# Strip the extension from game name
df["game"] = df["game"].map(lambda x: x.split(".")[0])

# Filterout game lurking and hollywood
#df = df[~df["game"].isin(["lurking", "hollywood"])]

# Add new column to order difficulty
df["order"] = df["Difficulty"].map({"Possible": 2, "Difficult": 1, "Extreme": 0})

df = df.sort_values(by=["order", "model", "Max Normalized Score"], ascending=True)

# Add new column color for each game based on the game difficulty
def color(game, diff):
    if diff == "Possible":
        return f"<span style='color:green'>{game}</span>"
    elif diff == "Difficult":
        return f"<span style='color:orange'>{game}</span>"
    elif diff == "Extreme":
        return f"<span style='color:red'>{game}</span>"
    else:
        raise ValueError("Unknown difficulty")

df["color"] = df.apply(lambda x: color(x["game"], x["Difficulty"]), axis=1)


fig = px.bar(df, x="Max Normalized Score", y="game", color="model", barmode="group", title="Max Normalized Score per game", height=1000)

# Sort the y axis by game difficulty and then by max score for gpt-4o
#fig.update_yaxes(categoryorder="array", categoryarray=df.sort_values(by=["order", "model", "Max Normalized Score"], ascending=True))
fig.update_yaxes(tickvals=df["game"], tickmode="array", ticktext=df["color"])

# Make x axis range from 0 to 100
fig.update_xaxes(range=[0, 100])

# sort game title by max score for gpt-4o
#fig.update_yaxes(categoryorder="total ascending")


/home/macote/miniconda3/envs/textworld/lib/python3.9/site-packages/plotly/express/_core.py:1992: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [292]:
runs_df[runs_df["game"] == "hollywood"]

,game,model,Max Normalized Score,Stochastic,Dialog,Darkness,Nonstandard Action,Inventory,Maze,Trivia,Self-Maintenance,Combat,Time,Mechanical Puzzle,Persistence,Difficulty,color,order
45,hollywood,meta-llama-3-70b-instruct-4,0.0,True,False,True,False,False,True,True,False,True,False,True,False,Difficult,<span style='color:orange'>hollywood</span>,1
72,hollywood,microsoft/Phi-3.5-mini-instruct,0.0,True,False,True,False,False,True,True,False,True,False,True,False,Difficult,<span style='color:orange'>hollywood</span>,1
22,hollywood,phi-3-medium-128k-instruct-1,0.0,True,False,True,False,False,True,True,False,True,False,True,False,Difficult,<span style='color:orange'>hollywood</span>,1


In [ ]:
# Create a group bar chart with the average score per game for each model
import plotly.express as px
import plotly.graph_objects as go

# sort by max score for gpt-4o
#runs_df = runs_df.sort_values(by=["model", "Max Normalized Score"], ascending=True)
# Strip the extension from game name
def color(text, diff):
    if diff == "Possible":
        s = '$\color{green}{' + str(text) + '}$'
    elif diff == "Difficult":
        s = '$\color{yellow}{' + str(text) + '}$'
    elif diff == "Extreme":
        s = '$\color{red}{' + str(text) + '}$'

    return s

fig.update_yaxes(tickvals=runs_df["game"], tickmode="array", ticktext=runs_df[["game", "Difficulty"]].map(lambda : color(game, diff)))
